In [ ]:
import requests # type: ignore
import numpy as np # type: ignore
import re

In [ ]:
NUM_COHORTS = 6 # 24
COHORT_SIZE = 4 # 8
NUM_ROUNDS = 3 # 75

MEMORY_SIZE = 10
MODEL_NAME= "mistral"

In [ ]:
TREATMENTS = {
	0.6: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 42),
		('B', 'R'): (42, 0),
		('B', 'B'): (12, 12)
	},
	1: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 40),
		('B', 'R'): (40, 0),
		('B', 'B'): (20, 20)
	},
	2: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 35),
		('B', 'R'): (35, 0),
		('B', 'B'): (40, 40)
	}
}

In [ ]:
API_URL = "https://grateful-glowing-hog.ngrok-free.app/"
GENERATE_ENDPOINT = "/ollama/api/generate"
CHAT_ENDPOINT = "/ollama/api/chat"

In [ ]:
rand_gen = np.random.default_rng()

In [ ]:
class Participant:
	def __init__(self, id):
		self.id = id
		self.memory = []
		self.payout = 0
		self.rounds = 0
		self.prompt = ""

	def __eq__(self, other):
		return self.id == other.id
	
	def set_prompt(self, prompt):
		self.prompt = prompt
	
	def update_memory(self, ctx):
		self.memory.append(ctx)
		if len(self.memory) >= MEMORY_SIZE:
			self.memory.pop(0)

In [ ]:

participant_ids = np.arange(1, NUM_COHORTS * COHORT_SIZE + 1, dtype=int)
rand_gen.shuffle(participant_ids)
# participants = np.array([(participant_id, np.empty((0,), dtype=list), 0) for participant_id in participant_ids], dtype=object)
participants = [Participant(id) for id in participant_ids]
cohorts = np.array_split(participants, NUM_COHORTS)

In [ ]:
split_cohorts = np.array_split(cohorts, len(TREATMENTS))
cohort_treatments = list(zip(split_cohorts, TREATMENTS.keys()))

In [ ]:
def prompt_model(prompt, context):
	print(f"Prompt: {prompt}")
	print(f"Context: {context}")
	messages = [
		*context,
		{
			"role": "user",
			"content": prompt
		}
	]
	data = {
    "model": MODEL_NAME,
		"messages": messages,
    "stream": False
	}
	print(f"Data: {data}")
	res = requests.post(API_URL + CHAT_ENDPOINT, json=data).json()['response']
	return [*messages, res]

In [ ]:
def get_choice(response):
	matches = re.findall("{[A-Z]}", response)
	if len(matches) > 0:
		return matches[-1]
	else:
		matches = re.findall("[RB]", response)
		if len(matches) > 0:
			return matches[-1]
		else:
			return ""

In [ ]:
def generate_start_prompt(treatment):
	return f"You are an undergraduate student participating in a lab experiment. You play a game with an anonymous player in which you simultaneously make a choice. Your payoff depends on both choices. If you both pick R, you each get {TREATMENTS[treatment][('R','R')][0]}$. If you choose R while they choose B, you get {TREATMENTS[treatment][('R','B')][0]}$, and they get {TREATMENTS[treatment][('R','B')][1]}$. Similarly, if you pick B while they pick R, you get {TREATMENTS[treatment][('B','R')][0]}$, and they get {TREATMENTS[treatment][('B','R')][1]}$. If you both pick B, you each earn {TREATMENTS[treatment][('B','B')][0]}$. The game has {NUM_ROUNDS} rounds. What's your choice? Perform reasoning as a human player. Append your choice letter in curly brackets as a last character."

In [ ]:
def generate_cont_prompt(choice1, choice2, treatment):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {treatment[(choice1, choice2)][0]}. Your opponent's payoff is {treatment[(choice1, choice2)][1]}. Please play the next round."

In [ ]:
def generate_end_prompt(choice1, choice2, treatment, payouts):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {treatment[(choice1, choice2)][0]}. Your opponent's payoff is {treatment[(choice1, choice2)][1]}. The game is now over. Your total payoff was {sum(payouts[0]) + treatment[(choice1, choice2)][0]}. Your opponent's total payoff was {sum(payouts[1]) + treatment[(choice1, choice2)][1]}."

In [ ]:
# for cohorts_for_treatment, treatment in cohort_treatments:
# 	for cohort in cohorts_for_treatment:
# 		for participant in cohort:
# 			p_memory = []
# 			o_memory = []
# 			for round in range(NUM_ROUNDS):
# 				print(participant)
# 				rand_participant = rand_gen.choice([x for x in cohort if x[0] != participant[0]])
# 				print(f"Treatment: {treatment} - Round: {round + 1} - Participant 1: {participant} - Participant 2: {rand_participant}")
# 				if round == 0:
# 					p_prompt = generate_start_prompt(treatment)
# 					o_prompt = generate_start_prompt(treatment)
# 					p_context = [*p_memory]
# 					o_context = [*o_memory]
# 				else:
# 					p_prompt = generate_start_prompt(treatment)
# 					o_prompt = generate_start_prompt(treatment)
# 					p_context = [*p_memory]
# 					o_context = [*o_memory]
				
# 				p_memory = prompt_model(p_prompt, p_context)
# 				o_memory = prompt_model(o_prompt, o_context)

In [ ]:
for cohorts_for_treatment, treatment in cohort_treatments:
	for cohort in cohorts_for_treatment:
		for round in range(NUM_COHORTS * COHORT_SIZE * NUM_ROUNDS // 2):
			participant, opponent = rand_gen.choice([x for x in cohort if x != participant and x.rounds < NUM_ROUNDS], size=2, replace=False)

			if participant.rounds == 0:
				participant.set_prompt(generate_start_prompt(treatment))
				ctx, res = prompt_model(participant.prompt, participant.memory)
				choice1 = get_choice(res)
				participant.update_memory(ctx)
			
			if opponent.rounds == 0:
				opponent.set_prompt(generate_start_prompt(treatment))
				ctx, res = prompt_model(opponent.prompt, opponent.memory)
				choice2 = get_choice(res)
				opponent.update_memory(ctx)

			participant.set_prompt(generate_cont_prompt(choice1, choice2, treatment))
			opponent.set_prompt(generate_cont_prompt(choice2, choice1, treatment))